In [2]:
import chardet
import re
import os
from glob import glob
import pandas as pd
import datetime
import collections
import random
from pandas_2_pptx.pandas_2_pptx import pandas_2_pptx

In [5]:
class Acpc():

    def __init__(self, dir_path, pptx_title):
        desc_list = []
        detail_list = []
        self.f_list = make_file_list(dir_path)
        for n,fname in enumerate(self.f_list):
            try:
                df = pd.read_csv(fname)
            except UnicodeDecodeError:
                df = pd.read_csv(fname,encoding='cp932')
            datetime_list = check_datetime_column(df)
            detail_list.append(self.__data_details(fname, datetime_list))
            desc = self.describe(df)    
            desc[["mean",	"std",	"min",	"1%",	"10%",	"50%",	"90%",	"99%",	"max"]] = desc[["mean",	"std",	"min",	"1%",	"10%",	"50%",	"90%",	"99%",	"max"]].astype(float).round(2)
            desc[["count"]] = desc[["count"]].astype(int)
            desc = desc.round(2)
            desc_list.append(desc)
        pptx = pandas_2_pptx()
        title_slide = pptx.add_title_slide(pptx_title)
        details = pd.concat(detail_list, axis=1, sort = False)
        details.columns = fname_list
        slide1 = pptx.add_slide("データ概要")
        slide1.add_table(details)
        slide1.generate()
        for n, i in enumerate(desc_list):
            slide2 = pptx.add_slide(f"基礎集計{fname_list[n]}")
            slide2.add_table(i)
            slide2.generate()
        pptx.save(f"{pptx_title}.pptx")



    def __delimiter_check(self, fname):
        """区切り文字をチェックする関数
        """
        data = open(fname, "r").read()
        try:
            df = pd.read_csv(fname)
        except UnicodeDecodeError:
            df = pd.read_csv(fname,encoding='cp932')
        splitter = 0
        split_list = []
        for n,rand in enumerate([random.randint(0, len(df)) for num in range(5)]):
            li = []
            for i in data.split("\n")[rand]:
                li.append(i)
            split_list.append([key for key, value in collections.Counter(li).items() if value == df.shape[1] - 1])
            try:
                delimiter = set(split_list[n-1]) & set(split_list[n]) & delimiter
            except:
                delimiter = set(split_list[n])
        if(len(delimiter) != 1):
            delimiter = None
        else:
            delimiter = list(delimiter)[0]
        return delimiter

    def __encoding_check(self, fname):
        """encodingをチェックする関数
        """
        data = open(fname, "rb")
        enc = chardet.detect(data.read())["encoding"]
        return enc

    def __data_details(self, fname, parse_dates=None):
        """csvデータを読み込んで、詳細情報を集計している関数
        """
        try:
            df = pd.read_csv(fname, parse_dates = parse_dates)
        except UnicodeDecodeError:
            df = pd.read_csv(fname,encoding='cp932', parse_dates = parse_dates)
        delimiter = self.__delimiter_check(fname)
        encoding = self.__encoding_check(fname)
        details_list = [
            delimiter,
            encoding,
            df.shape[0],
            df.shape[1],
        ]
        index_list = ["区切り文字","文字コード","行数","カラム数"]
        datetime_col = df.columns[(df.dtypes == '<M8[ns]').values]
        period_list = []
        for n, i in enumerate(datetime_col):
            index_list.append(f"データ期間[{i}]")
            period_list.append(f"{df[i].min()} ~ {df[i].max()}")
            details_list.append(period_list[n])
        return pd.Series(details_list, index = index_list)


In [20]:
def describe(data, percentiles=[0.01, 0.1, 0.9, 0.99],
             include='all', exclude=None, std_outlier=3):
    """
    各種統計量を算出.

    Args:
        data (pandas.DataFrame): 統計量を算出するデータ.
        percentiles (list-like of numbers): 分位点.
        include ('all', list-like of dtypes or None): 統計量を算出する変数.
        exclude (list-like of dtypes or None (default)): 統計量を算出しない変数.
        std_outlier (int): 外れ値を判定する際の係数.
            「平均 +- 標準偏差 × std_outlier」の範囲外のレコードを外れ値と判定する.

    Returns:
        desc (pandas.DataFrame): 各種統計量・外れ値の件数・欠損率・データ型.

    """
    desc = data.select_dtypes(exclude=["datetime"]).describe(percentiles=percentiles, include=include, exclude=exclude).T
    null_rate = (data.isnull().sum() / len(data)).to_frame('欠損率')
    dtypes = data.dtypes.to_frame('dtypes')
    data_int = data.select_dtypes(include=['int64', 'object'], exclude=[])
    unique = data_int.nunique().to_frame('unique')
    data_num = data.select_dtypes(['float', 'int'])
    desc_num = desc.loc[data_num.columns]
    lower = data_num < desc_num['mean'] - desc_num['std'] * std_outlier
    upper = data_num > desc_num['mean'] + desc_num['std'] * std_outlier
    lower = lower.sum().to_frame(f'mean-{std_outlier}std')
    upper = upper.sum().to_frame(f'mean+{std_outlier}std')
    #skew = data_num.skew().to_frame(f'歪度')
    #kurt = data_num.kurt().to_frame(f'尖度')

    args_merge = dict(left_index=True, right_index=True, how='left')
    desc = (
        desc
        .merge(null_rate, **args_merge)
        .merge(dtypes, **args_merge)
        #.merge(skew, **args_merge)
        #.merge(kurt, **args_merge)
    )
    desc["unique"] = unique
    return desc[['unique', 'count', '欠損率', 'mean', 'std', 'min', '1%', '10%', '50%', '90%', '99%', 'max', 'dtypes']]

In [19]:
def check_datetime_column(df):
    """
    Returns:
        df (pandas.DataFrame): object型の特定のカラムをdatetime型のカラムに変換したdataframe
        datetime_columns (list): datetime型に変換したカラム名のリスト
    """
    for column in df.select_dtypes(include='object').columns:
        try:
            df[column] = pd.to_datetime(df[column])
            print(i,"をdatetime型に変換しました")
        except:
            continue
    datetime_columns = df.select_dtypes(include='datetime64[ns]').columns.values
    print(datetime_columns)
    return df, datetime_columns

In [11]:
def make_file_list(path, f_type = "csv"):
    """
    Returns:
        f_list (list): ディレクトリ内のファイルリスト
    """
    f_list = glob(f'{path}{os.path.sep}**{os.path.sep}*.{f_type}',recursive=True)
    return f_list

In [5]:
fname_list = ["sample.csv", "purpose.csv"]
datetime_list = [["k", "l"], None]

In [6]:
%%time
faaaaa(fname_list,datetime_list,"test")

Wall time: 777 ms


In [29]:
test_df.dtypes

a           float64
b           float64
c           float64
d           float64
e           float64
f           float64
g           float64
h           float64
i           float64
j             int64
k    datetime64[ns]
l    datetime64[ns]
dtype: object

# 実験コード

In [28]:
from pptx import Presentation
from pptx.util import Inches

prs = Presentation()
title_only_slide_layout = prs.slide_layouts[5]
slide = prs.slides.add_slide(title_only_slide_layout)
shapes = slide.shapes

shapes.title.text = 'Adding a Table'

rows = cols = 2
left = top = Inches(2.0)
width = Inches(6.0)
height = Inches(0.8)

In [29]:
table = shapes.add_table(rows, cols, left, top, width, height).table

In [63]:
from pptx.util import Pt,Cm

In [32]:
# set column widths
table.columns[0].width = Inches(2.0)
table.columns[1].width = Inches(4.0)

# write column headings
run = table.cell(0, 0).text_frame.paragraphs[0].add_run()
run.text = 'Foo'
run.font.size = Pt(22)

table.cell(0, 1).text = 'Bar'

# write body cells
table.cell(1, 0).text = 'Baz'
table.cell(1, 1).text = 'Qux'

prs.save('test22222.pptx')

In [33]:
df = pd.read_csv("sample.csv")

In [62]:
def calculation_Pt(df,width_cm):
    column_num = df.shape[1] + 1
    return width_cm/column_num


In [65]:
Cm(0.035)

12600

In [66]:
Pt(1)

12700